<span id="menu"></span>
# Zipline Commission Models 手續費模型

> - 交易成本（transaction costs）被廣泛認為是影響投資績效的重要因素。它們不僅影響投資績效，還影響了將資產轉換成現金的難易度。
> - 在真實世界的交易中存在許多種類的交易成本，其中一種是**直接成本（direct cost）**。
> - 直接成本包含了**手續費（commission）、證交稅**等等，這些成本是在交易前已經**提前知道的成本**。
> - 此類成本皆可以使用 Zipline 中的 **commission models** 進行模擬。
> - 這也是回測（backtesting）的一大目的，考量投資策略在真實世界運行的可能性。
> 
> 
> ## Zipline 有三種 commission models：
> 
> 1. `PerShare`：按照下單的股數計算費用，同時還可以設定一個最低費用。
> 2. `PerTrade`：一筆交易收取一筆固定費用。
> 3. `PerDollar`：按照交易金額抽成計算。
> 4. `Custom_TW_Commission`：台灣專用的手續費模型。
>
> ## 閱讀本篇之前請先閱讀以下文章：
> 1. [TSMC buy and hold strategy.ipynb](https://github.com/tejtw/TQuant-Lab/blob/main/lecture/TSMC%20buy%20and%20hold%20strategy.ipynb) 
> 
> 2. [Zipline Order（order & order_target）.ipynb](https://github.com/tejtw/TQuant-Lab/blob/main/lecture/Zipline%20Order%20(order%20%26%20order_target).ipynb)
> 
> 3. [Zipline Order（value & target_value）.ipynb](https://github.com/tejtw/TQuant-Lab/blob/main/lecture/Zipline%20Order%20(value%20%26%20target_value).ipynb)
>
> 4. [Zipline Order（percent & target_percent）.ipynb](https://github.com/tejtw/TQuant-Lab/blob/main/lecture/Zipline%20Order%20(percent%20%26%20target_percent).ipynb)
>
> ## 閱讀本篇之後可以閱讀：
> 
> [Zipline Slippage.ipynb](https://github.com/tejtw/TQuant-Lab/blob/main/lecture/Zipline%20Slippage.ipynb)

## 設定方法
```python
from zipline.api import set_commission
from zipline.finance import commission

def initialize(context):
    set_commission(commission.<其中一種commission models>)
```

## 補充說明
- `set_commission`只能一次用**一種**方法。
- `set_commission`**只能**在`initialize`階段使用。
- 手續費計算時，**價格以成交日收盤價為準，數量也以成交時為準**，也就是說，如果因為股數變動造成 amount 有任何變化，計算上都是用成交時新的 amount。

## 設定環境

In [1]:
import pandas as pd
import numpy as np
import datetime
import tejapi
import time
import os
import warnings
warnings.filterwarnings('ignore')

# tej_key
tej_key = 'your key'
api_base = 'https://api.tej.com.tw'

os.environ['TEJAPI_KEY'] = tej_key
os.environ['TEJAPI_BASE']= api_base

tejapi.ApiConfig.api_key = tej_key  
tejapi.ApiConfig.api_base = api_base


# date
# set date
start='2022-12-01'
end='2022-12-31'
os.environ['mdate'] = '20221201 20221231'

tz = 'UTC'
start_dt, end_dt = pd.Timestamp(start, tz = tz), pd.Timestamp(end, tz = tz)

# calendar
calendar_name='TEJ'

# bundle_name
bundle_name = 'tquant'

# ticker
os.environ['ticker'] = '1216 IR0001'

In [2]:
!zipline ingest -b tquant

Merging daily equity files:


[2023-11-09 11:06:37.741495] INFO: zipline.data.bundles.core: Ingesting tquant.


In [3]:
from zipline.finance import commission, slippage
from zipline.api import *

from zipline import run_algorithm
from zipline.utils.calendar_utils import get_calendar

from zipline.utils.run_algo import (get_transaction_detail,
                                    get_record_vars)

## API Reference

### class zipline.finance.commission.<font color=DeepPink>PerDollar</font>(cost=0.0015)
按照交易金額抽成計算。

- cost：*float, optional*，每交易一元的股票所需支付的固定費用。預設為 0.0015 元。

### class zipline.finance.commission.<font color=DeepPink>PerTrade</font>(cost=0.0)
一筆交易收取一筆固定費用。

- cost：*float, optional*，每進行一筆交易所需支付的固定費用。預設為 0 元。

### class zipline.finance.commission.<font color=DeepPink>PerShare</font>(cost=0.001, min_trade_cost=0.0)
按照下單的股數計算費用，同時還可以設定一個最低費用。

- cost：*float, optional*，每交易一股的股票所需支付的固定費用。預設為 0.001 元。
- min_trade_cost：*float, optional*，最低費用，預設為無最低費用。

### class zipline.finance.commission.<font color=DeepPink>Custom_TW_Commission</font>(min_trade_cost=20, discount=1.0, tax = 0.003)
台灣適用的手續費模型，考量券商手續費（費率：0.001425）及證交稅，同時還可以設定一個最低費用。

- min_trade_cost：*float, optional*，最低費用。預設為 20 元。
- discount：*float, optional*，券商手續費折扣比率。預設為 1，代表沒有折扣。
- tax：*float, optional*，證交稅率，預設為 0.003。

## 範例：比較四種計算方法，注意四個initialize函數的差別

In [4]:
def initialize_perdollar(context):
    context.i = 0
    context.tickers = ['1216']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    
    # set_commission
    set_commission(commission.PerDollar(cost=0.001))
    
    set_benchmark(symbol('IR0001'))
    
def initialize_pertrade(context):
    context.i = 0
    context.tickers = ['1216']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    
    # set_commission
    set_commission(commission.PerTrade(cost=0.5))
    
    set_benchmark(symbol('IR0001'))
    
def initialize_pershare(context):
    context.i = 0
    context.tickers = ['1216']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    
    # set_commission
    set_commission(commission.PerShare(cost=0.001, min_trade_cost=5.0))
    
    set_benchmark(symbol('IR0001'))
    
def initialize_Custom(context):
    context.i = 0
    context.tickers = ['1216']
    context.asset = [symbol(ticker) for ticker in context.tickers]      
    set_slippage(slippage.FixedSlippage(spread=0.00))
    
    # set_commission
    set_commission(commission.Custom_TW_Commission(min_trade_cost=20,
                                                   discount = 1.0,
                                                   tax = 0.003))
    
    set_benchmark(symbol('IR0001'))
    
def handle_data(context, data):

    if context.i == 0:
        for asset in context.asset:
            order_target(asset, 1000)
    if context.i == 2:
        for asset in context.asset:
            order_target(asset, 0)
            
    record(close=data.current(context.asset, 'close'))
    context.i += 1

capital_base = 1e5

In [5]:
closing_price = tejapi.get('TWN/APIPRCD',
                           coid=['1216'],
                           opts={'columns':['mdate','coid','close_d']},
                           mdate={'gte':start_dt,'lte':end_dt },
                           paginate=True)

perdollar = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize_perdollar,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

pertrade = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize_pertrade,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

pershare = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize_pershare,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

Custom = run_algorithm(start=start_dt,
                            end=end_dt,
                            initialize=initialize_Custom,
                            handle_data=handle_data,
                            capital_base=capital_base,
                            trading_calendar=get_calendar(calendar_name),
                            bundle=bundle_name)

## 講解說明

## 12/1

In [6]:
# 收盤價
closing_price[0:2]

,mdate,coid,close_d
None,,,
0,2022-12-01,1216,65.9
1,2022-12-02,1216,65.0


### PerDollar
同樣在12/1下單一張統一（1216）股票，如果用 PerDollar 算法，費用就是下一個交易日12/2的收盤價 65 * 1000 股 * 0.001 = 65。

In [7]:
# PerDollar算法：費用65元
perdollar['orders'][1]

[{'id': '6cead32ce3df4e9eb68b82cab2dfd20e',
  'dt': Timestamp('2022-12-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 65.0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

### PerTrade
如果是 PerOrder，就固定是預先設定好的 0.5 元。

In [8]:
# pertrade算法：費用0.5元
pertrade['orders'][1]

[{'id': '919bf42bb0ff40dd8cbddd1f0b711de4',
  'dt': Timestamp('2022-12-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 0.5,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

### PerShare
PerShare 雖然設定一股抽 0.001，但是因為 1000 股 * 0.001 = 1 小於最低費用 min_trade_cost = 5，所以費用是 5。

In [9]:
# pershare算法：費用5元
pershare['orders'][1]

[{'id': '872a709b04fa4419ab9033c1c63d5cf1',
  'dt': Timestamp('2022-12-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 5.0,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

### Custom_TW_Commission

## 台灣的情況
在台灣交易股票時，主要有兩個直接成本：**手續費**及**證交稅**。`Custom_TW_Commission`可模擬這兩項成本。

### 手續費
- **買進**或**賣出**時皆須繳交。
- 計算方式為：成交價 × 成交股數 × 0.1425 % × 折扣（折扣預設是=1，沒有折扣）。
- 手續費有**最低價格**（預設是=20元）。

### 證交稅
- **賣出**時才要繳交。
- 計算方式為：成交價 × 成交股數 × 證交稅率（證交稅率預設是0.3%）。

In [10]:
# 買進：65 * 1000 股 * 0.001425 = 93
Custom['orders'][1]

[{'id': '6d7307629ab84512b42272b250afa6d6',
  'dt': Timestamp('2022-12-02 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-01 13:30:00+0800', tz='Asia/Taipei'),
  'amount': 1000,
  'filled': 1000,
  'commission': 93,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

In [11]:
# 賣出：64.6 * 1000 股 * 0.001425 + 64.6 * 1000 股 * 0.003 = 93 + 194 = 287
Custom['orders'][3]

[{'id': 'f2bf52f2c0ac435398c06d1ffda180b1',
  'dt': Timestamp('2022-12-06 13:30:00+0800', tz='Asia/Taipei'),
  'reason': None,
  'created': Timestamp('2022-12-05 13:30:00+0800', tz='Asia/Taipei'),
  'amount': -1000,
  'filled': -1000,
  'commission': 287,
  'stop': None,
  'limit': None,
  'stop_reached': False,
  'limit_reached': False,
  'sid': Equity(0 [1216]),
  'status': <ORDER_STATUS.FILLED: 1>}]

[Return to Menu](#menu)